# T81-558: Applications of Deep Neural Networks
### TensorFlow (SKFLOW) Meets KDD-99
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

This simple example shows how to load a non-trivial dataset from CSV and train a neural network.  The dataset is the
[KDD99 dataset](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html).  This dataset is used to detect between normal and malicious network activity.


# Content Moved

The content previously at this page has moved to: [KDD-99 Keras](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class12_security.ipynb)